# Ethical Reminder (from the Lab Alliance Compact)

- Data belongs to truth, not expectations; document steps transparently.
- Perform **your own analysis**; credit all sources and collaborators properly.
- Communicate respectfully; ask for help early; uphold psychological safety.

> By proceeding, you acknowledge the Compact and agree to act accordingly.


## Gaussian peak fitting pilot

**Learning goals**
1. Generate synthetic Gaussian data with known ground truth.
2. Perform nonlinear fit using `scipy.optimize.curve_fit`.
3. Interpret parameter uncertainties.
4. Diagnose fits with residuals.
5. Understand systematic vs. statistical uncertainty in peak fitting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

plt.style.use('seaborn-v0_8-colorblind')


## Generate synthetic data

We assume the observable is described by a Gaussian with an amplitude, center, width, and constant background.


In [ ]:
rng = np.random.default_rng(2024)

def gaussian(x, amplitude, mean, sigma, offset):
    return amplitude * np.exp(-0.5 * ((x - mean) / sigma) ** 2) + offset

x = np.linspace(-5, 5, 120)
true_params = {
    'amplitude': 3.5,
    'mean': 0.6,
    'sigma': 0.9,
    'offset': 0.25,
}
sigma_obs = np.full_like(x, 0.15)
y_clean = gaussian(x, **true_params)
noise = rng.normal(0.0, sigma_obs)
y = y_clean + noise


## Visualize the data

Plot the simulated measurements alongside the noise-free model to see the scale of the fluctuations.


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.errorbar(x, y, yerr=sigma_obs, fmt='o', markersize=4, capsize=3, label='noisy data')
ax.plot(x, y_clean, color='black', lw=2, label='true model')
ax.set_xlabel('x')
ax.set_ylabel('Signal')
ax.set_title('Synthetic Gaussian data with constant background')
ax.legend()
plt.tight_layout()
plt.show()


## Fit the Gaussian model

`curve_fit` returns both the best-fit parameters and the covariance matrix.
Passing `sigma` and `absolute_sigma=True` treats the supplied σ values as standard deviations of each point.


In [ ]:
p0 = (3.0, 0.0, 1.0, 0.0)  # rough guesses for amplitude, mean, sigma, offset
popt, pcov = curve_fit(gaussian, x, y, p0=p0, sigma=sigma_obs, absolute_sigma=True)
perr = np.sqrt(np.diag(pcov))

param_names = ('amplitude', 'mean', 'sigma', 'offset')
for name, value, err in zip(param_names, popt, perr):
    truth = true_params[name]
    print(f"{name:9s} = {value:6.3f} ± {err:6.3f}  (true: {truth:6.3f})")

residuals = y - gaussian(x, *popt)
chi2 = np.sum((residuals / sigma_obs) ** 2)
ndof = x.size - len(popt)
print(f"chi^2 = {chi2:.1f}, ndof = {ndof}, chi^2/ndof = {chi2/ndof:.2f}")


## Residual analysis

Residuals help diagnose structure that the model misses. Random scatter about zero suggests an adequate model.


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6), sharex=True)
model_y = gaussian(x, *popt)
ax1.errorbar(x, y, yerr=sigma_obs, fmt='o', markersize=4, capsize=3, label='data')
ax1.plot(x, model_y, color='tab:red', label='fit')
ax1.set_ylabel('Signal')
ax1.set_title('Best-fit Gaussian model')
ax1.legend()

ax2.axhline(0.0, color='black', lw=1)
ax2.errorbar(x, residuals, yerr=sigma_obs, fmt='o', markersize=4, capsize=3)
ax2.set_xlabel('x')
ax2.set_ylabel('Residuals')
ax2.set_title('Residuals with 1σ error bars')
plt.tight_layout()
plt.show()


In [ ]:
# Explore sensitivity to outliers and systematic offsets
    y_outlier = y.copy()
    outlier_index = np.argmax(model_y)
    y_outlier[outlier_index] += 1.0  # introduce a single large spike

    y_offset = y + 0.2  # mimic an uncorrected background shift

    popt_outlier, _ = curve_fit(gaussian, x, y_outlier, p0=popt, sigma=sigma_obs, absolute_sigma=True)
    popt_offset, _ = curve_fit(gaussian, x, y_offset, p0=popt, sigma=sigma_obs, absolute_sigma=True)

    print('Parameter shifts from a single outlier:')
    for name, base, pert in zip(param_names, popt, popt_outlier):
        print(f"  {name:9s}: {base:6.3f} → {pert:6.3f}")

    print('
Parameter shifts from a +0.2 systematic offset:')
    for name, base, pert in zip(param_names, popt, popt_offset):
        print(f"  {name:9s}: {base:6.3f} → {pert:6.3f}")


### Interpreting results
- χ² close to the number of degrees of freedom indicates the point-by-point uncertainties are realistic.
- Correlated parameters (e.g., amplitude vs. offset) can inflate uncertainties; inspect `pcov`.
- Outliers disproportionately affect the fit: robust estimators or data-quality flags may be necessary.
- A global offset shifts the background term without fixing the underlying cause—verify calibrations and subtraction steps.


## Exercises (short)
1. **Noise level**: Change `sigma_obs` and regenerate the data. How do parameter errors scale with noise?
2. **Weighted vs unweighted**: Repeat the fit with `sigma=None`. What happens to χ² and parameter uncertainties?
3. **Outlier handling**: Remove or down-weight the injected outlier in the sensitivity cell. How do the parameters respond?
4. **Systematic offset**: Model the +0.2 offset explicitly (e.g., by fitting an additional baseline parameter informed by calibration data). What experimental cross-checks would reveal such a bias?
